In [84]:

import os
import re
import glob
import datetime
import warnings
import pandas as pd
import numpy as np
from fuzzywuzzy import process

import sys
import warnings
sys.path.insert(0, '../') 
# from utils import init_logger

def common_subheaders()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        ('Advertising, Public Relations and Marketing ',
        'Air Transportation',
        'Amusement and Recreation',
        'Apparel Manufacturing',
        'Building Equipment Contractors',
        'Business Support Services',
        'Chemicals',
        'Communications Equipment Manufacturing',
        'Credit Related Activities',
        'Computer Systems Design and Related Services',
        'Credit (Nondepository)',
        'Data Processing and Hosting Services',
        'Educational Support Services',
        'Electronic Component Manufacturing',
        'Equipment Leasing',
        'Facilities Support Services',
        'Grocery Stores',
        'Hospitals',
        'Insurance',
        'Lessors of Nonfinancial Licenses',
        'Management, Scientific, and Technical Consulting Services',
        'Motion Picture and Video Industries',
        'Other Information Services',
        'Other Manufacturing',
        'Other Publishing',
        'Other Real Estate Activities',
        'Other Telecommunications',
        'Plastics Manufacturing',
        'Radio and Television Broadcasting',
        'Real Estate Leasing',
        'Restaurants',
        'Retail',
        'Satellite Telecommunications',
        'Scientific Research and Development Services',
        'Texttile Furnishings Mills',
        'Traveler Arrangement',
        'Software Publishing',
        'Utility System Construction',
        'Wholesalers',
        'Wired Telecommunications Carriers',
        'Wireless Telecommunications Carriers',
        )
    ))


def standard_field_names()->tuple:
    return (
        'Portfolio Company',
        # 'Portfolio Company /Principal Business',
        'Investment /Interest Rate /Maturity',
        'Percentage  Interest/  Shares',
        'Principal',
        'Cost',
        'Value',
        'Percent of Class Held',
        'Investment',
        'CDO Fund Investments',
        'Percent of Interests Held',
        'Industry',
        'Spread Above Index',
        'Aquisition Date',
        'Interest Rate',
        'Maturity',
        'Principal/Shares',
        'Investment Type',
        'of Net Assets',
        'business description',
        'type of investment',
        'investment date',
        'reference rate and spread',
        'pik rate',
        'maturity date',
        'cost',
        'footnotes',
        'industry',
        'principal amount', # TODO change stand names for more dynamic fuzzywuzzy matching
        'fair value',
    )

    
def company_control_headers()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        (
        'Debt Investments',
        'Debt Investments (82.23%)',
        'Debt Investments (A)',
        'Debt Investments (continued)',
        'Equity Securities',
        'Equity Securities (continued)',
        'Cash and Cash Equivalents',
        )
    ))

def exceptions()->dict:
    return {
        '2006-12-31\\Schedule_of_Investments_1.csv':dict(),
        '2006-12-31\\Schedule_of_Investments_3.csv':dict(), 
        '2008-03-31\\Schedule_of_Investments_7.csv':dict(),
        '2008-03-31\\Schedule_of_Investments_8.csv':dict(),
        '2008-12-31\\Schedule_of_Investments_6.csv':dict(),
        '2008-12-31\\Schedule_of_Investments_11.csv':dict(),
        '2008-12-31\\Schedule_of_Investments_13.csv':dict(),
        '2008-12-31\\Schedule_of_Investments_14.csv': {
                    'Portfolio_Company_/Principal_Business': np.array([ True, False, False, False, False, False, False, False, False,False, False, False, False, False,False]),
                    '': np.array([ True, False,  True, True, False, True,  True, True, False,True,  True, True, False, True,False]),
                    'Investment': np.array([ True, False, False, False, False, False]),
                    'Percent_of_Interests_Held': np.array([ True,  False,  False, False, False, False]),
                    'Cost': np.array([True,  False, False, False, False, False]),
                    'Value': np.array([True,  False, False, False, False, False])
                },
        '2009-03-31\\Schedule_of_Investments_2.csv':dict(),
        '2009-03-31\\Schedule_of_Investments_5.csv':dict(),
        '2009-03-31\\Schedule_of_Investments_6.csv':dict(),
        '2009-03-31\\Schedule_of_Investments_9.csv':dict(),
        '2009-09-30\\Schedule_of_Investments_8.csv':dict(),
        '2009-12-31\\Schedule_of_Investments_6.csv':dict(),
        '2009-12-31\\Schedule_of_Investments_7.csv':dict(),
        '2009-12-31\\Schedule_of_Investments_12.csv': {
            'Portfolio Company': np.array([ True, False, False, False, False, False, False, False, False,False, False, False, False, False]),
            'Investment /Interest Rate /Maturity': np.array([ False, True, False, False, False, False, False, False, False,False, False, False, False, False]),
            'Percent_of_Interests_Held':np.array([False,False,True,False,False,False,False,False,False,False,False,False,False,False]),
            'Cost':np.array([False,False,False,False,True,True,False,False,False,False,False,False,False,False]),
            'Value':np.array([False,False,False,False,False,False,True,True,False,False,False,False,False]),
            '':np.array([True]*6 + [False]*6),
        },
        '2010-03-31\\Schedule_of_Investments_7.csv':dict(),
        '2010-03-31\\Schedule_of_Investments_8.csv':dict(),
        '2010-06-30\\Schedule_of_Investments_19.csv':dict(),
        '2011-06-30\\Schedule_of_Investments_4.csv':dict(),
        '2011-06-30\\Schedule_of_Investments_6.csv':dict(),
        '2011-06-30\\Schedule_of_Investments_9.csv':dict(),
        '2011-12-31\\Schedule_of_Investments_9.csv':dict(),
        '2012-03-31\\Schedule_of_Investments_27.csv':dict()

    }

def except_rows()->tuple:
    return (
        'asdf',
    )

# https://www.sec.gov/robots.txt
def get_standard_name(col, choices, score_cutoff=60):
    best_match, score = process.extractOne(col, choices)
    if score > score_cutoff:
        return best_match
    return col

def stopping_criterion(qtr:str)->str:
    return '{}'.format(r'Total\s*Investments')

 
def concat(*dfs)->list:
    final = []
    for df in dfs:
        final.extend(df.values.tolist())
    return final

    
def get_key_fields(
    df_cur:pd.DataFrame,
)->tuple:
    important_fields = standard_field_names() + common_subheaders()
    for idx,row in enumerate(df_cur.iterrows()):
        found = any(any(
            key in str(field).lower() 
            for key in important_fields)
                    for field in row[-1].dropna().tolist()
            )
        if found and len(set(row[-1].dropna().tolist())) >= 6:
            cols = df_cur.iloc[:idx + 1].apply(lambda row: ' '.join(row.dropna()), axis=0).tolist()
            fields = strip_string(cols,standardize=found) 
            return fields
    return strip_string(df_cur.iloc[0].tolist())

def strip_string(
    columns_names:list,
    standardize:bool=False
)->tuple:
    # columns = tuple(map(lambda col:re.sub(r'[^a-z]', '', str(col).lower()),columns_names))
    if standardize:
        standard_fields = standard_field_names()
        return tuple(
            re.sub(r'\s+', '_',get_standard_name(str(col),standard_fields)) for col in columns_names
        )
    return tuple(re.sub(r'\s+', '_',str(col)) for col in columns_names)


# Function to extract date and convert to datetime object
def extract_date(file_path):
    # Extract date from file path (assuming date is always in 'YYYY-MM-DD' format)
    date_str = re.search(r'\d{4}-\d{2}-\d{2}', file_path).group()
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')


def merge_duplicate_columns(
    df:pd.DataFrame,
    merged_pair_idxs:dict={}
)->pd.DataFrame:
    duplicate_cols = merged_pair_idxs.keys()
    flag = not merged_pair_idxs.keys()
    if flag: 
        duplicate_cols = df.columns.unique() 
    for col_name in duplicate_cols:
        # display(col_name)
        mask = merged_pair_idxs.get(col_name)
        if flag:
            mask = df.columns == col_name
            merged_pair_idxs[col_name] = mask
        duplicate_data = df.loc[:, mask]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, ~mask]
        df[col_name] = merged_data
        # display(df)
    return df.reset_index(drop=True),merged_pair_idxs

def extract_subheaders(
    df:pd.DataFrame,
    control:bool,
)->pd.DataFrame:
    col_name = 'company_control' if control else 'Type_of_Investment'
    if col_name in df.columns:
        return df
    include = df.apply(
        lambda row: re.search('|'.join(company_control_headers() if control else common_subheaders()), str(row[0]), re.IGNORECASE) is not None,
        axis=1
    )  
    
    exclude = ~df.apply(
        lambda row: row.astype(str).str.contains('total|Inc|Ltd|LLC|Holdings|LP|Co|Corporation', case=False, na=False).any(),
        axis=1
    )
    idx = df[include & exclude].index.tolist()
    df[col_name] = None
    if not idx:
        return df

    prev_header = subheader = None
    df.loc[idx[-1]:,col_name] = df.iloc[idx[-1],1] if isinstance(df.iloc[idx[-1],0],float)  else df.iloc[idx[-1],0]
    for j,i in enumerate(idx[:-1]):
        prev_header = subheader
        subheader = df.iloc[i,1] if isinstance(df.iloc[i,0],float)  else df.iloc[i,0]
        df.loc[idx[j]:idx[j+1],col_name] = subheader if subheader != '' else prev_header
    return df


def remove_row_duplicates(row:pd.Series)->pd.Series: 
    out = []
    for v in row:
        if v in out and not str(v).replace('$','').isnumeric():
            out.append(np.nan)
        else:
            out.append(v)
    return pd.Series(out)

  

def _clean(
    file_path:str,
    except_rows:str,
    merged_pair_idxs:dict={},
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0,na_values=[' ', ''])
    # df.replace(['\xa0','\u200b',r'^\s$',r'^\s%'],np.nan,regex=True,inplace=True) #':','$','%' r'^\s*$',r'^\s*%'
    df.replace(['\xa0','\u200b',r'^\s*$',r'^\s*%',' ','˄'],'',regex=True,inplace=True)
    df.replace(['PrincipalBusiness'],'',regex=True,inplace=True)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.dropna(axis=0,how='all',inplace=True)
    df = df[~df.apply(lambda row:row.astype(str).str.contains(except_rows,case=False, na=False).any(),axis=1)]
    if df.shape[0] < 3:
        return pd.DataFrame(), merged_pair_idxs
    if not merged_pair_idxs:
        important_fields = strip_string(get_header_rows(df),standardize=True)#get_key_fields(df)
        df.columns = important_fields
    df,merge_pair_idxs = merge_duplicate_columns(df,merged_pair_idxs=merged_pair_idxs)
    duplicate_idx = df.apply(lambda row:row[pd.to_numeric(row,errors='coerce').isna()].duplicated().sum() > 1 ,axis=1)
    clean_rows = df.loc[duplicate_idx].apply(remove_row_duplicates, axis=1).reset_index(drop=True)
    j = 0
    for i,flag in enumerate(duplicate_idx):
        if not flag:
            continue
        df.iloc[i,:] = clean_rows.loc[j,:]
        j += 1
    df.replace([''],np.nan,regex=True,inplace=True) #':','$','%'
    df.dropna(axis=1,how='all',inplace=True)
    
    columns = (~df.isna()).sum(axis=0) <= (4  if df.shape[0] > 12 else 2 if df.shape[0] == 4 else 0)
    df = df.drop(columns=df.columns[columns])
    return df.reset_index(drop=True),merge_pair_idxs



def get_header_rows(
    df_cur:pd.DataFrame,
)->tuple:
    for idx,row in df_cur.iterrows():
        found = any(str(v).replace("$",'').replace("%",'').isnumeric() for v in row)
        if found:
            out = df_cur.iloc[:idx,:].apply(lambda row: ' '.join(row[row.notna()].values), axis=0)
            return out
    
    return strip_string(df_cur.iloc[0].tolist())


def main()->None:
    qtrs = os.listdir('.')
    ex = exceptions()
    ex_rows = '|'.join(except_rows())
    for qtr in qtrs:
        if '.csv' in qtr or not os.path.exists(os.path.join(qtr,f'Schedule_of_Investments_0.csv')):
            continue
        qtr = '2020-12-31'
        # logger.info(qtr)
        display(qtr)
        index_list_sum = i = 0
        soi_files = sorted([
            os.path.join(qtr,file) 
            for file in os.listdir(qtr)
            if file.endswith('.csv')
        ],key=lambda f: int(f.split('_')[-1].split('.')[0]))
        # soi_files = [f for f in soi_files] # if f not in ex]
        df,merged_pair_idxs = _clean(soi_files[i],except_rows=ex_rows,merged_pair_idxs={})

        index_list = df.apply(
            lambda row:row.astype(str).str.contains(stopping_criterion(qtr), case=False, na=False).any(),
            axis=1
        )
        index_list_sum = index_list.sum()
        dfs = [df]     
        i += 1

        while index_list_sum == 0:
            # logger.info(soi_files[i])
            merged_pair_idxs = ex.get(soi_files[i],merged_pair_idxs)
            display(soi_files[i])

            # display(merged_pair_idxs)
            df,merged_pair_idxs = _clean(soi_files[i],except_rows=ex_rows,merged_pair_idxs=merged_pair_idxs if soi_files[i] not in ex else {})
            dfs.append(df)
            index_list = df.apply(
                lambda row:row.astype(str).str.contains(stopping_criterion(qtr), case=False, na=False).any(),
                axis=1
            )
            index_list_sum = index_list.sum()
            i += 1
        date_final = dfs[0]
        if len(dfs) > 1:
            date_final = pd.concat(dfs,axis=0,ignore_index=True)#pd.DataFrame(concat(*dfs))
        # date_final = extract_subheaders(date_final,control=True)
        # date_final = extract_subheaders(date_final,control=False)

        date_final['qtr'] = qtr.split('\\')[-1]
        if not os.path.exists(os.path.join(qtr,'output')):
            os.makedirs(os.path.join(qtr,'output'))
        columns_to_drop = date_final.notna().sum() <= 2
        date_final.drop(columns=columns_to_drop[columns_to_drop].index)
        date_final.to_csv(os.path.join(qtr,'output',f'{qtr}.csv'),index=False)
        break
    
    # Use glob to find files
    files = sorted(glob.glob(f'*/output/*.csv'), key=extract_date)
    single_truth = pd.concat([
        pd.read_csv(df) for df in files
    ],axis=0,ignore_index=True)
    single_truth.drop(columns=single_truth.columns[['Unnamed' in col for col in single_truth.columns]],inplace=True)
    single_truth.to_csv(f'{cik}_soi_table.csv',index=False)
    
warnings.simplefilter(action='ignore', category=FutureWarning)
cik = 1372807
# logger = init_logger(cik)
# logger.info(cik)

main()


'2020-12-31'

'2020-12-31\\Schedule_of_Investments_1.csv'

'2020-12-31\\Schedule_of_Investments_2.csv'

'2020-12-31\\Schedule_of_Investments_3.csv'

'2020-12-31\\Schedule_of_Investments_4.csv'

...

'2020-12-31\\Schedule_of_Investments_42.csv'

'2020-12-31\\Schedule_of_Investments_43.csv'

'2020-12-31\\Schedule_of_Investments_44.csv'

'2020-12-31\\Schedule_of_Investments_45.csv'

'2020-12-31\\Schedule_of_Investments_46.csv'

In [82]:
file_path = r'2018-12-31\\Schedule_of_Investments_4.csv'



def standard_field_names()->tuple:
    return (
        'Portfolio Company',
        # 'Portfolio Company /Principal Business',
        'Percentage  Interest/  Shares',
        'Investment /Interest Rate /Maturity',        
        'Principal',
        'Cost',
        'Value',
        'Percent of Class Held',
        'Investment',
        'CDO Fund Investments',
        'Percent of Interests Held',
        'Industry',
        'Spread Above Index',
        'Aquisition Date',
        'Interest Rate',
        'Maturity',
        'Principal/Shares',
        'Investment Type',
        'of Net Assets',
        'business description',
        'type of investment',
        'investment date',
        'reference rate and spread',
        'pik rate',
        'maturity date',
        'cost',
        'footnotes',
        'industry',
        'principal amount', # TODO change stand names for more dynamic fuzzywuzzy matching
        'fair value',
    )

def merge_duplicate_columns(
    df:pd.DataFrame,
    merged_pair_idxs:dict={}
)->pd.DataFrame:
    duplicate_cols = merged_pair_idxs.keys()
    flag = not merged_pair_idxs.keys()
    if flag: 
        duplicate_cols = df.columns.unique() 
    for col_name in duplicate_cols:
        # display(col_name)
        mask = merged_pair_idxs.get(col_name)
        if flag:
            mask = df.columns == col_name
            merged_pair_idxs[col_name] = mask
        duplicate_data = df.loc[:, mask]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, ~mask]
        df[col_name] = merged_data
        # display(df)
    return df.reset_index(drop=True),merged_pair_idxs

def _clean(
    file_path:str,
    except_rows:str,
    merged_pair_idxs:dict={},
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0,na_values=[' ', ''])
    # df.replace(['\xa0','\u200b',r'^\s$',r'^\s%'],np.nan,regex=True,inplace=True) #':','$','%' r'^\s*$',r'^\s*%'
    df.replace(['\xa0','\u200b',r'^\s*$',r'^\s*%',' ','˄'],'',regex=True,inplace=True)
    df.replace(['PrincipalBusiness'],'',regex=True,inplace=True)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.dropna(axis=0,how='all',inplace=True)
    df = df[~df.apply(lambda row:row.astype(str).str.contains(except_rows,case=False, na=False).any(),axis=1)]
    if df.shape[0] < 3:
        return pd.DataFrame(), merged_pair_idxs
    if not merged_pair_idxs:
        important_fields = strip_string(get_header_rows(df),standardize=True)#get_key_fields(df)
        df.columns = important_fields
    df,merge_pair_idxs = merge_duplicate_columns(df,merged_pair_idxs=merged_pair_idxs)
    duplicate_idx = df.apply(lambda row:row[pd.to_numeric(row,errors='coerce').isna()].duplicated().sum() > 1 ,axis=1)
    clean_rows = df.loc[duplicate_idx].apply(remove_row_duplicates, axis=1).reset_index(drop=True)
    j = 0
    for i,flag in enumerate(duplicate_idx):
        if not flag:
            continue
        df.iloc[i,:] = clean_rows.loc[j,:]
        j += 1
    df.replace([''],np.nan,regex=True,inplace=True) #':','$','%'
    df.dropna(axis=1,how='all',inplace=True)
    
    columns = (~df.isna()).sum(axis=0) <= (4  if df.shape[0] > 12 else 2 if df.shape[0] == 4 else 0)
    df = df.drop(columns=df.columns[columns])
    return df.reset_index(drop=True),merge_pair_idxs


merged_pair_idxs = {}
ex = exceptions()
ex_rows = '|'.join(except_rows())
df,merged_pair_idxs = _clean(file_path,except_rows=ex_rows,merged_pair_idxs=merged_pair_idxs)
display(df)
display(merged_pair_idxs)
index_list = df.apply(
    lambda row:row.astype(str).str.contains(stopping_criterion(None), case=False, na=False).any(),
    axis=1
)
index_list_sum = index_list.sum()
index_list_sum

,Portfolio_Company,Investment,Aquisition_Date,Principal/Shares,Cost,Value
0,PortfolioCompany/,Investment(15),InitialAcquisitionDate,PercentageOwnership/Shares,Cost,FairValue(2)
1,"AdvancedLightingTechnologies,Inc,(8)(13)Consum...",Warrants,6/13/2012,1.90%,$ —,$ 1000
2,"AdvancedLightingTechnologies,Inc.(8)(13)Consum...",MembershipInterests,6/13/2012,0.40%,181999,1000
3,AerostructuresHoldingsL.P.(8)(13)AerospaceandD...,PartnershipInterests,2/28/2007,1.16%,157717,50000
4,BCPGreatLakesFundLP(17)LimitedPartnership,PartnershipInterest,12/11/2018,55.6%,12466667,12466667
5,CaribeMediaInc.(fkaCaribeInformationInvestment...,Common,12/18/2006,1.17%,359765,108675
6,"eInstructionAcquisition,LLC(8)(13)Services:Bus...",MembershipUnits,7/2/2007,1.10%,1079617,1000
7,"FPWRCACoinvestmentFundVII,Ltd.(3)(13)CapitalEq...",ClassAShares,2/2/2007,0.41%,1500000,669000
8,"NewMillenniumHoldco,Inc.(MillenniumHealth,LLC)...",Common,10/7/2014,0.20%,1953299,1000
9,"PerseusHoldingCorp.(8)(13)Hotel,Gaming&Leisure",Common,4/5/2007,0.19%,400000,1000


{'Portfolio_Company': array([ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False]),
 '': array([ True,  True, False,  True,  True, False, False, False, False,
         True,  True, False, False, False, False,  True,  True, False,
        False, False, False,  True,  True, False, False, False, False,
         True, False]),
 'Investment': array([ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]),
 'Aquisition_Date': array([ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]),
 'Principal/Shares': array([ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False]),


0

In [29]:
pd.read_csv('2015-03-31\\Schedule_of_Investments_1.csv')

,Unnamed: 0,0
0,0,5
